In [16]:
# Identifying Consumer Segments (Python)
# prepare for Python version 3x features and functions
from __future__ import division, print_function
# import packages for multivariate analysis
import pandas as pd  # DataFrame structure and operations
import numpy as np  # arrays and numerical processing
from sklearn.cluster import KMeans  # cluster analysis by partitioning
from sklearn.metrics import silhouette_score as silhouette_score

# read data from comma-delimited text file... create DataFrame object
bank = pd.read_csv('bank.csv', sep = ';')
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [2]:
# look at the list of column names
list(bank.columns.values)

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'response']

In [3]:
# examine the demographic variable age
print(bank['age'].unique())
print(bank['age'].value_counts(sort = True))
print(bank['age'].describe())  

[30 33 35 59 36 39 41 43 20 31 40 56 37 25 38 42 44 26 55 67 53 68 32 49
 78 23 52 34 61 45 48 57 54 63 51 29 50 27 60 28 21 58 22 46 24 77 75 47
 70 65 64 62 66 19 81 83 80 71 72 69 79 73 86 74 76 87 84]
34    231
32    224
31    199
36    188
33    186
35    180
37    161
38    159
30    150
40    142
42    141
41    135
39    130
46    119
43    115
48    114
45    112
49    112
47    108
44    105
28    103
50    101
29     97
27     94
53     94
57     91
51     91
55     90
52     86
58     85
     ... 
23     20
61     16
22      9
66      9
63      8
70      7
21      7
64      7
62      7
65      6
77      6
69      6
71      6
80      6
75      6
73      6
67      5
19      4
83      4
72      4
79      4
20      3
74      3
78      3
68      2
76      2
84      1
81      1
86      1
87      1
Name: age, Length: 67, dtype: int64
count    4521.000000
mean       41.170095
std        10.576211
min        19.000000
25%        33.000000
50%        39.000000
75%        49.000000
ma

In [4]:
# examine the demographic variable job
print(bank['job'].unique())
print(bank['job'].value_counts(sort = True))
print(bank['job'].describe())  

['unemployed' 'services' 'management' 'blue-collar' 'self-employed'
 'technician' 'entrepreneur' 'admin.' 'student' 'housemaid' 'retired'
 'unknown']
management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: job, dtype: int64
count           4521
unique            12
top       management
freq             969
Name: job, dtype: object


In [5]:
# define job indicator variables
job_indicators = pd.get_dummies(bank['job'], prefix = 'job')
job_indicators.head()

,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown
0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0


In [6]:
bank = bank.join(job_indicators)
bank['whitecollar'] = bank['job_admin.'] + bank['job_management'] + \
    bank['job_entrepreneur'] + bank['job_self-employed']
bank['bluecollar'] = bank['job_blue-collar'] + bank['job_services'] + \
    bank['job_technician'] + bank['job_housemaid']

In [7]:
# examine the demographic variable marital
print(bank['marital'].unique())
print(bank['marital'].value_counts(sort = True))
print(bank['marital'].describe())  
# define marital indicator variables
marital_indicators = pd.get_dummies(bank['marital'], prefix = 'marital')
marital_indicators.head()

['married' 'single' 'divorced']
married     2797
single      1196
divorced     528
Name: marital, dtype: int64
count        4521
unique          3
top       married
freq         2797
Name: marital, dtype: object


,marital_divorced,marital_married,marital_single
0,0,1,0
1,0,1,0
2,0,0,1
3,0,1,0
4,0,1,0


In [8]:
bank = bank.join(marital_indicators)
bank['divorced'] = bank['marital_divorced']
bank['married'] = bank['marital_married']

In [9]:
# examine the demographic variable education
print(bank['education'].unique())
print(bank['education'].value_counts(sort = True))
print(bank['education'].describe())  

# define education indicator variables
education_indicators = pd.get_dummies(bank['education'], prefix = 'education')
print(education_indicators.head())
bank = bank.join(education_indicators)
bank['primary'] = bank['education_primary']
bank['secondary'] = bank['education_secondary']
bank['tertiary'] = bank['education_tertiary']

print(bank.head)  # check the structure of the data frame
print(bank.shape)

['primary' 'secondary' 'tertiary' 'unknown']
secondary    2306
tertiary     1350
primary       678
unknown       187
Name: education, dtype: int64
count          4521
unique            4
top       secondary
freq           2306
Name: education, dtype: object
   education_primary  education_secondary  education_tertiary  \
0                  1                    0                   0   
1                  0                    1                   0   
2                  0                    0                   1   
3                  0                    0                   1   
4                  0                    1                   0   

   education_unknown  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  
<bound method NDFrame.head of       age            job   marital  education default  balance housing loan  \
0      30     unemployed   married    primary      no     1787      no   no   
1      33       services 

In [10]:
# look at the list of column names
list(bank.columns.values)

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'response',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'whitecollar',
 'bluecollar',
 'marital_divorced',
 'marital_married',
 'marital_single',
 'divorced',
 'married',
 'education_primary',
 'education_secondary',
 'education_tertiary',
 'education_unknown',
 'primary',
 'secondary',
 'tertiary']

In [11]:
# select/filter for cases never previously contacted by sales
bank_selected = bank[bank['previous'] == 0]
print(bank_selected.shape)

(3705, 43)


In [12]:
# select subset of variables needed for cluster analysis and post-analysis
bankfull = pd.DataFrame(bank_selected, \
    columns = ['response', 'age', 'whitecollar', 'bluecollar', 
               'divorced', 'married',
               'primary', 'secondary', 'tertiary'])

# examine the structure of the full bank DataFrame
print(bankfull.head)  # check the structure of the data frame
print(bankfull.shape)
# look at the list of column names
list(bankfull.columns.values)

<bound method NDFrame.head of      response  age  whitecollar  bluecollar  divorced  married  primary  \
0          no   30            0           0         0        1        1   
3          no   30            1           0         0        1        0   
4          no   59            0           1         0        1        0   
7          no   39            0           1         0        1        0   
8          no   41            1           0         0        1        0   
10         no   39            0           1         0        1        0   
11         no   43            1           0         0        1        0   
12         no   36            0           1         0        1        0   
13        yes   20            0           0         0        0        0   
15         no   40            1           0         0        1        0   
16         no   56            0           1         0        1        0   
18         no   25            0           1         0        0        

['response',
 'age',
 'whitecollar',
 'bluecollar',
 'divorced',
 'married',
 'primary',
 'secondary',
 'tertiary']

In [20]:
bankfull.head()

,response,age,whitecollar,bluecollar,divorced,married,primary,secondary,tertiary,cluster
0,no,30,0,0,0,1,1,0,0,0
3,no,30,1,0,0,1,0,0,1,0
4,no,59,0,1,0,1,0,1,0,1
7,no,39,0,1,0,1,0,1,0,0
8,no,41,1,0,0,1,0,0,1,0


In [22]:
# select subset of variables for input to cluster analysis
data_for_clustering = pd.DataFrame(bank_selected, columns = ['age', 'whitecollar', 'bluecollar', 
               'divorced', 'married','primary', 'secondary', 'tertiary'])
               
# convert to matrix/numpy array for input to cluster analysis
data_for_clustering_matrix = data_for_clustering.as_matrix()            
  
# investigate alternative numbers of clusters using silhouette score
silhouette_value = []
k = range(2,21)  # look at solutions between 2 and 20 clusters
for i in k:
    clustering_method = KMeans(n_clusters = i, random_state = 9999)
    clustering_method.fit(data_for_clustering_matrix)
    labels = clustering_method.predict(data_for_clustering_matrix)
    silhouette_average = silhouette_score(data_for_clustering_matrix, labels)
    silhouette_value.append(silhouette_average)    
              
# highest silhouette score is for two clusters
# so we use that clustering solution here   
clustering_method = KMeans(n_clusters = 2, random_state = 9999)
clustering_method.fit(data_for_clustering_matrix)
labels = clustering_method.predict(data_for_clustering_matrix)

# add cluster labels to bankfull and review the solution 
bankfull['cluster'] = labels

# pivot table and cross-tabulation examples
bankfull.pivot_table(['cluster'],index=['age'])  
pd.crosstab(bankfull.cluster, bankfull.bluecollar, margins = True)

C:\Users\bradl\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


bluecollar,0,1,All
cluster,,,
0,1079,1224,2303
1,756,646,1402
All,1835,1870,3705


In [23]:
# groupby example
segments = bankfull.groupby('cluster')
segments.describe()

age                                                    bluecollar  \
          count       mean       std   min   25%   50%   75%   max      count   
cluster                                                                         
0        2303.0  34.293964  5.064226  19.0  31.0  34.0  38.0  43.0     2303.0   
1        1402.0  52.236805  6.489732  44.0  47.0  51.0  56.0  87.0     1402.0   

                  ...  tertiary      whitecollar                           \
             mean ...       75%  max       count      mean       std  min   
cluster           ...                                                       
0        0.531481 ...       1.0  1.0      2303.0  0.405992  0.491190  0.0   
1        0.460770 ...       0.0  1.0      1402.0  0.369472  0.482834  0.0   

                             
         25%  50%  75%  max  
cluster                      
0        0.0  0.0  1.0  1.0  
1        0.0  0.0  1.0  1.0  

[2 rows x 64 columns]